In [12]:
import pandas as pd

data = [['name1',36,125000],['name2',28,75000],['name3',32,100000]]
df = pd.DataFrame(data, columns = ['Name','Age','Salary'])

#transform the given distribution to a distribution with mean 0 and standard deviation as 1
#tranform the given data between the range of 0 and 1 based on the min and max values of given input range
#tranform the given data between the range of -1 and 1 by dividing the value with maximum value


from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler,Normalizer

cols = df.select_dtypes(include=['float64','int64','int']).columns
df,scaler = scale_dataframe(df,StandardScaler(),cols,'standard')
#df,scaler = scale_dataframe(df,MinMaxScaler(),cols,'minmax')
#df,scaler = scale_dataframe(df,Normalizer(),cols,'normalize')
#df,scaler = scale_dataframe(df,MaxAbsScaler(),cols,'maxabs')
df.head()

,Name,Age,Salary,Age_standard_scaled,Salary_standard_scaled
0,name1,36,125000,1.224745,1.224745
1,name2,28,75000,-1.224745,-1.224745
2,name3,32,100000,0.000000,0.000000


In [13]:

def scale_dataframe(df,Scaler,cols,func):
    scaler = Scaler.fit(df[cols])
    df = pd.concat([df,pd.DataFrame(scaler.transform(df[cols]),columns=cols+'_'+func+'_scaled')],axis=1)
    return df,scaler


In [14]:
print(scaler.inverse_transform(df[cols+'_standard_scaled']))

[[3.60e+01 1.25e+05]
 [2.80e+01 7.50e+04]
 [3.20e+01 1.00e+05]]


In [ ]:
# Use scikit-learn to grid search the batch size and epochs
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD

# Function to create model, required for KerasClassifier

def create_model(learn_rate, momentum):
    model = Sequential()
    model.add(Dense(12, input_dim=1, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    model.compile(loss='binary_crossentropy', optimizer= optimizer, metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)


# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [16, 32, 64, 128, 256, 512, 1024]
epochs = [10, 25, 50, 75, 100, 150, 200, 300, 400, 500, 750, 1000, 1250, 1500, 1750, 2000]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

param_grid = dict(batch_size=batch_size, epochs=epochs,learn_rate=learn_rate, momentum=momentum)

#param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

X = df['Age']
Y = df['Salary']

grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Using TensorFlow backend.
C:\Users\jawad_zf1uaw5\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
